In [57]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification, BertConfig, BertModel
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.utils import class_weight
import numpy as np
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import json
import random
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import re

In [32]:
pd.options.display.max_colwidth = 3000
pd.options.display.max_rows = 3000

In [33]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cuda')
device

device(type='cuda')

In [34]:
#data = pd.read_csv('./Backup master data/master_data_6.csv', usecols=['ADR', 'text', 'category', 'type', 'date'])
# data = pd.read_csv('./oldnew.csv')
# data = pd.read_csv('/data/mmortgage/amal_workspace/multipageAccuracy/data/raw/master_data_V3_0_19Sept22.csv')
data = pd.read_csv('/data/mmortgage/amal_workspace/multipageAccuracy/data/processed/master_data_V3_0_20Sept22_processed_sampledc1.csv')
data.shape

(146435, 12)

In [35]:
data['type'].value_counts()

LD      116631
NMIC     29804
Name: type, dtype: int64

In [36]:
data.isnull().sum()

ADR                     0
text                    0
category                0
type                    0
date                    0
ogr/aug                 0
page#                   0
angle                   0
renamed category    68381
text_processed          0
word_count              0
category_new            0
dtype: int64

In [37]:
data.dropna(subset=['text'], inplace=True)
data.shape

(146435, 12)

In [38]:
data['category'].value_counts()

Bank Statement                                        16392
Tax Return - Business                                 15794
Purchase Agreement                                    15150
Tax Return - Personal                                 13349
Paystub                                               13033
Hazard Insurance Dec Page - Initial                   12688
Retirement Account Statement(s)                       10958
W-2(s) / 1099(s)                                       6539
Condominium Documents                                  5730
Rental Agreements(s)                                   5268
Mortgage Statement                                     5043
Driver's License                                       3475
Divorce Decree / Child Support                         3292
Title Report                                           3158
Divorce Decree                                         2806
Escrow/Closing Instructions                            1993
K1s                                     

In [39]:
# n = data[data['category']=='Bank Statement'].index.tolist()
# data.drop(index= random.sample(n, 10000), inplace=True)

In [40]:
# data['category'].value_counts()

In [41]:
mappings = {'Assets - Source of funds': 'Bank Statement',
 'Bank Statement': 'Bank Statement',
 'Bank Statement(s)': 'Bank Statement',
 'BK Papers': 'Bankruptcy Papers',
 'Child Support/Alimony Agreement': 'Divorce Decree / Child Support',
 'Construction Agreement/Contract': 'Purchase Agreement',
 'Contractor Bids': 'Purchase Agreement',
 'Contractor Docs': 'Purchase Agreement',
 'Escrow/Closing Instructions' : 'Escrow / Closing Instructions',
 'Divorce Decree': 'Divorce Decree / Child Support',
 'Earnest Money Deposit': 'Purchase Agreement',
 'Gift Funds': 'Bank Statement',
 'Hazard Insurance Contact Information': 'Hazard Insurance',
 'Hazard Insurance Dec Page - Final': 'Hazard Insurance',
 'Hazard Insurance Dec Page - Incomplete': 'Hazard Insurance',
 'Hazard Insurance Dec Page - Initial': 'Hazard Insurance',
 'Hazard Insurance Dec Page - Insufficient Coverage': 'Hazard Insurance',
 'Hazard Insurance Declaration': 'Hazard Insurance',
 'Insurance - Cost Estimator': 'Hazard Insurance',
 'Insurance - Hazard Checklist': 'Hazard Insurance',
 'Insurance - Proof Paid': 'Hazard Insurance',
 'Insurance Dec Page, Other': 'Hazard Insurance',
 'Insurance Document - Other': 'Hazard Insurance',
 'Investment Account Statements': 'Retirement Account Statement(s)',
 'Mortgage Statement': 'Mortgage Statement',
 'Mortgage Statement of Inspection of Subject Property': 'Mortgage Statement',
 'Mortgage Statement/Coupon, 1st': 'Mortgage Statement',
 'Mortgage Statement/Coupon, 2nd': 'Mortgage Statement',
 'Other Property 1st Mtg Stmt': 'Mortgage Statement',
 'Other Property 2nd Mtg Stmt': 'Mortgage Statement',
 'Purchase Agreement': 'Purchase Agreement',
 'Purchase Agreement Addendum': 'Purchase Agreement',
 'Purchase Document Other': 'Purchase Agreement',
 'Rental Agreements(s)': 'Rental Agreements(s)',
 'Retirement Account Statement(s)': 'Retirement Account Statement(s)',
 'Sales Checklist': 'Purchase Agreement',
 'Sales Contract and Addendums': 'Purchase Agreement',
 'Sales Contract/Purchase Agreement': 'Purchase Agreement',
 'Sales Forms': 'Purchase Agreement',
 'Sales Stips': 'Purchase Agreement',
 'Divorce Decree / Child Support': 'Divorce Decree / Child Support',
 "Non-Mortgage Account Statement" : "Non-Mortgage Statement"}

In [42]:
data['category'] = data['category'].replace(mappings)

In [43]:
data['category'].value_counts()

Bank Statement                                        16392
Tax Return - Business                                 15794
Purchase Agreement                                    15401
Tax Return - Personal                                 13349
Paystub                                               13033
Hazard Insurance                                      12688
Retirement Account Statement(s)                       10958
W-2(s) / 1099(s)                                       6539
Divorce Decree / Child Support                         6098
Condominium Documents                                  5730
Rental Agreements(s)                                   5268
Mortgage Statement                                     5043
Driver's License                                       3475
Title Report                                           3158
Escrow / Closing Instructions                          2382
K1s                                                    1670
Award Letter(s)                         

### Stopword removal

In [44]:
# alterted_text=[]
# remove_words = set(stopwords.words('english') + list(string.punctuation))
# for text in data.text.values:
#     text = re.sub(r'[~^0-9]', '', text)
#     wordlist = [word for word in word_tokenize(text.lower()) if not word in remove_words]
#     alterted_text.append((" ").join(wordlist))

In [45]:
# data['text']= alterted_text

### Training

In [46]:
temp = []

cats = ['Retirement Account Statement(s)',
        'Mortgage Statement',
        'Bank Statement',
        'Non-Mortgage Statement',
        'Due Diligence Report',
        'Purchase Agreement',
        'Rental Agreements(s)',
        'Hazard Insurance',
        'Divorce Decree / Child Support',
        'Bankruptcy Papers']

# cats = ['Retirement Account Statement(s)',
#         'Mortgage Statement',
#         'Bank Statement',
#         'Non-Mortgage Statement']

        
        
for category in data['category']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append("focused")
data['new_category'] = temp

In [47]:
data['new_category'].unique()

array(['others', 'focused'], dtype=object)

In [48]:
label_dict = {}
label_dict['focused'] = 1
label_dict['others0'] = 0
label_dict

{'focused': 1, 'others0': 0}

In [49]:
# possible_labels = data.new_category.unique()

# label_dict = {}
# label_dict['others']=0
# for i,cat in enumerate(cats):
#     label_dict[cat]=i+1
# label_dict

In [50]:
data['label'] = data.new_category.replace(label_dict)
# data.head(1)

In [51]:
df = data.sample(frac=0.80, random_state=200)
df.shape

(117148, 14)

In [56]:
df['category'].value_counts()

Bank Statement                                        13083
Tax Return - Business                                 12638
Purchase Agreement                                    12257
Tax Return - Personal                                 10741
Paystub                                               10475
Hazard Insurance                                      10115
Retirement Account Statement(s)                        8714
W-2(s) / 1099(s)                                       5260
Divorce Decree / Child Support                         4918
Condominium Documents                                  4605
Rental Agreements(s)                                   4235
Mortgage Statement                                     4016
Driver's License                                       2809
Title Report                                           2509
Escrow / Closing Instructions                          1907
K1s                                                    1337
Award Letter(s)                         

In [52]:
test = data.drop(df.index).reset_index(drop=True)
test.shape

(29287, 14)

In [53]:
# df.to_csv("/data/mmortgage/amal_workspace/multipageAccuracy/data/train dataset/train_set_20_SEPT_2022.csv", index=False)

In [54]:
test['new_category'].value_counts()

focused    14937
others     14350
Name: new_category, dtype: int64

In [55]:
df['new_category'].value_counts()

focused    58986
others     58162
Name: new_category, dtype: int64

In [ ]:
df['new_category'].unique().tolist()

In [ ]:
classweight = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                             classes=df['new_category'].unique().tolist(),
                                                             y=df['new_category'].values.tolist()), dtype=torch.float)
classweight

In [ ]:
df.reset_index(drop=True, inplace=True)
# df.head(1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = 'not_set'
df.head(2)

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['new_category', 'label', 'data_type']).count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased-customizedtokens')
# '../amal_workspace/multipageAccuracy/bert-base-uncased'

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_processed.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
batch_size=32
dataloader_train = DataLoader(dataset_train, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   shuffle=False,
                                   num_workers=0, 
                                   batch_size=batch_size)

In [ ]:
len(dataloader_train)

In [ ]:
len(dataloader_train)*32

In [58]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, 
                 dropout=0.1, 
                 BERTconfig=False, 
                 attention_heads=12, 
                 hidden_layers=12, 
                 numclass=1,
                 hidden_dropout_prob=0.1,
                 hidden_act='gelu',
                 position_embedding_type='absolute'):
        
        #self.attention_heads = attention_heads
        super(BertClassifier, self).__init__()
        
        if BERTconfig:
            configuration = BertConfig(num_attention_heads= attention_heads, 
                                       num_hidden_layers= hidden_layers)   
        else:
            configuration = BertConfig()
        #self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=attention)
        #print(configuration)
        self.bert = BertModel(configuration)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, numclass, bias=True)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

In [59]:
label_dict

{'focused': 1, 'others0': 0}

In [60]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)


model = BertClassifier(dropout=0.3, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.3,
                       position_embedding_type='absolute')

model.to(device)
model.train()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
EPOCH = 2

save_model = 250

criterion = nn.CrossEntropyLoss(weight=classweight.to(device), reduction='mean')

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=250,
                                            num_training_steps=len(dataloader_train)*EPOCH)


logs = []
i=0

for epoch in range(EPOCH):
    
    print('epoch--- ',epoch+1)
    
    for train_input in tqdm(dataloader_train):
        i=i+1
        model.zero_grad()
        input_id = train_input[0].to(device)
        mask = train_input[1].to(device)
        train_label = train_input[2].to(device)
        output = model(input_id, mask)
        batch_loss = criterion(output, train_label)
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        
        if(i%save_model==0):
            label_train =  torch.tensor([]).to(device)
            pred_train = torch.tensor([]).to(device)
            total_loss_train = 0

            for train_eval in dataloader_train:
                input_id = train_eval[0].to(device)
                mask = train_eval[1].to(device)
                train_label = train_eval[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                label_train = torch.cat((label_train, train_label), 0)
                pred_train = torch.cat((pred_train, output.argmax(dim=1)), 0)
            
            label_train = label_train.detach().cpu().numpy()
            pred_train = pred_train.detach().cpu().numpy()
            label_train = label_train.astype("int32")
            pred_train = pred_train.astype("int32")
            total_acc_train = accuracy_score(label_train, pred_train)
#             print(total_acc_train)
            total_f1_train = f1_score(label_train, pred_train, average='weighted')
            total_loss_train = total_loss_train/(len(dataloader_train)*batch_size)
            

            
            label_val =  torch.tensor([]).to(device)
            pred_val = torch.tensor([]).to(device)
            total_loss_val = 0

            for val_input in dataloader_validation:
                input_id = val_input[0].to(device)
                mask = val_input[1].to(device)
                val_label = val_input[2].to(device)
                output = model(input_id, mask)
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                label_val = torch.cat((label_val, val_label), 0)
                pred_val = torch.cat((pred_val, output.argmax(dim=1)), 0)
            
            
            label_val = label_val.detach().cpu().numpy()
            pred_val = pred_val.detach().cpu().numpy()
            label_val = label_val.astype("int32")
            pred_val = pred_val.astype("int32")
            total_acc_val = accuracy_score(label_val, pred_val)
            total_f1_val = f1_score(label_val, pred_val, average='weighted')
            total_loss_val = total_loss_val/(len(dataloader_validation)*batch_size)
            
            print('train_accuracy: ', total_acc_train, ', train_f1: ', total_f1_train, ', train_loss: ', total_loss_train)
            print('val_accuracy: ', total_acc_val, ',  val_f1: ', total_f1_val, ', val_loss: ', total_loss_val)

            torch.save(model.state_dict(), 
                       f'/mnt/bert_models/BERT_encoder{HIDDEN_LAYER}_attention{ATTENTION}_epoch{i}.model')

            logs.append({'epoch': i,
                     'train_accuracy': total_acc_train,
                     'train_f1': total_f1_train,
                     'train_loss': total_loss_train,
                     'val_accuracy': total_acc_val,
                     'val_f1': total_f1_val,
                     'val_loss': total_loss_val})

In [ ]:
log = pd.DataFrame(logs)
log['val_loss'] = log['val_loss'].round(3)
log['train_loss'] = log['train_loss'].round(3)
log['train_f1'] = log['train_f1'].round(2)
log['val_f1'] = log['val_f1'].round(2)
#log.index = log.epoch
#del log['epoch']
log

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(20,10))
ax.grid(True)
plt1 = ax.plot(log.epoch, log.train_f1, color="red", label= 'train_f1')
plt2 = ax.plot(log.epoch, log.val_f1, color="green", label= 'val_f1')

# set x-axis label
ax.set_xlabel("iterations", fontsize = 14)

# set y-axis label
ax.set_ylabel("F1", fontsize=14)

ax.set_xticks(log.epoch.values)

ax2=ax.twinx()
plt3 = ax2.plot(log.epoch, log.train_loss,color="red", label='train_loss')
plt4 = ax2.plot(log.epoch, log.val_loss,color="green", label= 'val_loss')

lns = plt1+plt2+plt3+plt4
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='center right')

ax2.set_ylabel("LOSS", fontsize=14)
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
#plt.plot(log.epoch, log.train_accuracy, label= 'train_accuracy')
plt.plot(log.epoch, log.train_f1, label= 'train_f1')
#plt.plot(log.epoch, log.val_accuracy, label= 'val_accuracy')
plt.plot(log.epoch, log.val_f1, label='val_f1')
plt.legend(loc="upper left")
plt.xticks(log.epoch, rotation=90)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(log.epoch, log.train_loss, label='train_loss')
plt.plot(log.epoch, log.val_loss, label='val_loss')
plt.legend(loc="upper right")
plt.xticks(log.epoch, rotation=90)
plt.grid(True)
plt.show()

In [ ]:
'''EPOCH = 5

criterion = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=500,
                                            num_training_steps=len(dataloader_train)*EPOCH)

logs = []

for epoch in range(EPOCH):
    
    print('epoch--- ',epoch+1)
    total_loss_train = 0
    label_train =  torch.tensor([], dtype=torch.uint8).to(device)
    pred_train = torch.tensor([], dtype=torch.uint8).to(device)

    for train_input in tqdm(dataloader_train):
        model.zero_grad()
        input_id = train_input[0].to(device)
        mask = train_input[1].to(device)
        train_label = train_input[2].to(device)
        output = model(input_id, mask)
        label_train = torch.cat((label_train, train_label), 0)
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        pred_train = torch.cat((pred_train, output.argmax(dim=1)), 0)
        batch_loss.backward()
        optimizer.step()
        scheduler.step()
        #break
    
    label_train = label_train.detach().cpu().numpy()
    pred_train = pred_train.detach().cpu().numpy()
    total_acc_train = accuracy_score(label_train, pred_train)
    total_f1_train = f1_score(label_train, pred_train, average='weighted')
    total_loss_train = total_loss_train/(len(dataloader_train)*batch_size)

    label_val =  torch.tensor([], dtype=torch.uint8).to(device)
    pred_val = torch.tensor([], dtype=torch.uint8).to(device)
    total_loss_val = 0

    with torch.no_grad():
        for val_input in tqdm(dataloader_validation):
            input_id = val_input[0].to(device)
            mask = val_input[1].to(device)
            val_label = val_input[2].to(device)
            output = model(input_id, mask)
            label_val = torch.cat((label_val, val_label), 0)
            batch_loss = criterion(output, val_label)
            total_loss_val += batch_loss.item()
            pred_val = torch.cat((pred_val, output.argmax(dim=1)), 0)
            #break
    
    label_val = label_val.detach().cpu().numpy()
    pred_val = pred_val.detach().cpu().numpy()
    total_acc_val = accuracy_score(label_val, pred_val)
    total_f1_val = f1_score(label_val, pred_val, average='weighted')
    total_loss_val = total_loss_val/(len(dataloader_validation)*batch_size)
    
    torch.save(model.state_dict(), f'./model/BERT_encoder{HIDDEN_LAYER}_attention{ATTENTION}_epoch{epoch+1}.model')
    
    print('train_accuracy: ', total_acc_train, ', train_f1: ', total_f1_train, ', train_loss: ', total_loss_train)
    print('val_accuracy: ', total_acc_val, ',  val_f1: ', total_f1_val, ', val_loss: ', total_loss_val)
    print('\n')
    
    logs.append({'epoch': epoch+1,
                 'train_accuracy': total_acc_train,
                 'train_f1': total_f1_train,
                 'train_loss': total_loss_train,
                 'val_accuracy': total_acc_val,
                 'val_f1': total_f1_val,
                 'val_loss': total_loss_val})'''

In [ ]:
label_dict

### Evaluation

In [ ]:
# df_test['label'] = df_test.new_category.replace(label_dict)
#df_test = pd.read_csv("/data/mmortgage/amal_workspace/multipageAccuracy/data/processed/testing_domain_processed.csv")
test.head(2)

In [ ]:
test.shape

In [ ]:
test['category'].value_counts()

In [ ]:
# df_test["new_category"] = "others1"
# df_test["new_category"].loc[df_test["Actual Classification"] == "Non-Mortgage Statement"] = "Non-Mortgage Statement"
# df_test["new_category"].loc[df_test["Actual Classification"] == "Bank Statement"] = "Bank Statement"
# df_test["new_category"].loc[df_test["Actual Classification"] == "Mortgage Statement"] = "Mortgage Statement"
# df_test["new_category"].loc[df_test["Actual Classification"] == "Retirement Account Statement(s)"] = "Retirement Account Statement(s)"

In [ ]:
test['new_category'].value_counts()

In [ ]:
def BERTEncode(df, size, shuffle=False):
    encoded_data = tokenizer.batch_encode_plus(
        df.text_processed.values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        max_length=512, 
        truncation=True,
        return_tensors='pt')
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(df.label.values)
    tensordataset = TensorDataset(input_ids, attention_masks, labels)
    dataloader = DataLoader(tensordataset,
                            shuffle=shuffle,
                            num_workers=0,
                            batch_size=size)
    return dataloader

In [ ]:
tokenizer = BertTokenizer.from_pretrained('./bert-base-uncased-customizedtokens') # '../amal_workspace/multipageAccuracy/bert-base-uncased'
dataloader_test = BERTEncode(test, 32)

In [ ]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)

model = BertClassifier(dropout=0.2, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.1,
                       position_embedding_type='absolute')
#model = BertClassifier(BERTconfig=False, numclass= CLASSES)

model.to(device)

model.load_state_dict(torch.load("/mnt/bert_models/BERT_encoder1_attention12_epoch1750.model", map_location=torch.device('cuda')))
model.eval()

In [ ]:
def validate(dataloader, model):
    actual =  torch.tensor([]).to(device)
    predicted = torch.tensor([]).to(device)
    cr = []

    for input in tqdm(dataloader):
        input_id = input[0].to(device)
        mask = input[1].to(device)
        labels = input[2].to(device)
        actual = torch.cat((actual, labels), 0)
        with torch.no_grad():
            output = model(input_id, mask)
        predicted = torch.cat((predicted, output.argmax(dim=1)), 0)
        for logits in output:
            score = torch.nn.functional.softmax(logits, dim=0)
            cr.append(np.float64(score.max(dim=0)[0].detach().cpu().numpy()))
    
    actual = actual.detach().cpu().numpy()
    predicted = predicted.detach().cpu().numpy()
    actual = actual.astype('int8')
    predicted = predicted.astype('int8')
    total_acc_test = accuracy_score(actual, predicted)
    total_f1_test = f1_score(actual, predicted, average='weighted')
    print('Test Accuracy:', round(total_acc_test,2))
    print('Test f1:', round(total_f1_test,2))
    return (actual, predicted, cr)

In [ ]:
actual, predicted, cr =validate(dataloader_test, model)

In [ ]:
label_dict_inv=dict()
for label in label_dict:
    label_dict_inv[label_dict[label]]=label
label_dict_inv 

In [ ]:
checkfor = range(len(label_dict))
#checkfor = [0]
final_predicted = []
    
classes = []
for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p, c in zip(actual, predicted, cr):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            elif (p!=check):
                notclassified+=1
                score.append(c)
                if(check==1):
                    classes.append(p)
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_dict_inv[check],correct,total,acc))
        print('notclassified {}'.format(notclassified))
        if notclassified!=0:
            print('notclassified score for {}'.format(np.median(score)))
    print('\n')

In [ ]:
np.unique(classes, return_counts=True)

### GLOBAL_VALIDATION

In [ ]:
global_validate = pd.read_csv('./global_validate.csv')
global_validate.head(2)

In [ ]:
#global_validate = global_validate[global_validate['new_text_format']=='new']

In [ ]:
global_validate['category'].value_counts()

In [ ]:
#global_validate.drop(global_validate[global_validate['category']=='Tax Return'].index, inplace=True)

In [ ]:
count=0
index=[]
for i,row in global_validate.iterrows():
    if row['ADR'] in data['ADR'].values.tolist():
        if row['text'] in data['text'][data['ADR']==row['ADR']].values.tolist():
            count+=1
            index.append(i)
count

In [ ]:
#del global_validate['new_category']
#global_validate.drop(index=index, inplace=True)
global_validate.shape

In [ ]:
#global_validate['category'].value_counts()

In [ ]:
global_validate.to_csv('./global_validate.csv', header=True, index=False)

In [ ]:
temp = []
# cats = ['Retirement Account Statement(s)',
#         'Mortgage Statement',
#         'Bank Statement',
#         'Purchase Agreement',
#         'Rental Agreements(s)',
#         'Hazard Insurance',
#         'Divorce Decree / Child Support',
#         'Bankruptcy Papers']

cats = ['Purchase Agreement',
        'Rental Agreements(s)',
        'Due Diligence Report']

for category in global_validate['category']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append(category)
global_validate['new_category'] = temp
global_validate.head(2)

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate['label'] = global_validate.new_category.replace(label_dict)
global_validate.head(1)

In [ ]:
global_validate['text']= [str(text) for text in global_validate['text']]

In [ ]:
def BERTEncode(df, size, shuffle=False):
    encoded_data = tokenizer.batch_encode_plus(
        df['new_format_text'].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        max_length=512, 
        truncation=True,
        return_tensors='pt')
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(df.label.values)
    tensordataset = TensorDataset(input_ids, attention_masks, labels)
    dataloader = DataLoader(tensordataset,
                            shuffle=shuffle,
                            num_workers=0,
                            batch_size=size)
    return dataloader

In [ ]:
dataloader = BERTEncode(global_validate, 32, False)

In [ ]:
ATTENTION = 12
HIDDEN_LAYER = 1
CLASSES = len(label_dict)


model = BertClassifier(dropout=0.2, 
                       BERTconfig=True, 
                       attention_heads=ATTENTION,
                       hidden_layers=HIDDEN_LAYER,
                       numclass= CLASSES,
                       hidden_dropout_prob=0.1,
                       position_embedding_type='absolute')

model.to(device)
model.load_state_dict(torch.load('/mnt/bert_models/BERT_encoder1_attention12_epoch5250.model', map_location=torch.device('cuda')))
model.eval()

In [ ]:
actual, predicted, cr = validate(dataloader, model)

In [ ]:
global_validate.shape

In [ ]:
label_dict_inv=dict()
for label in label_dict:
    label_dict_inv[label_dict[label]]=label
label_dict_inv 

In [ ]:
label_dict

In [ ]:
threshholds = range(0,92,2)
checkfor = range(len(label_dict))

accuracy=[]
notclassified = []
for thresh in threshholds:
    
    final_predicted=[]
    for a,p,c in zip(actual, predicted, cr):
        if c>(thresh/100):
            final_predicted.append(p)
        else:
            final_predicted.append(0)
    
    acc = 0
    incorrect = 0
    for check in checkfor:
        total = 0
        correct = 0
        for a,p,c in zip(actual, final_predicted, cr):
            if a == check:
                total+=1
                if (p == check):
                    correct+=1
                elif (p!=check):
                    incorrect+=1
        if(total!=0):
            acc = acc+round(correct/total,2)
    #accuracy.append(acc/len(checkfor))
    accuracy.append(acc/len(global_validate['new_category'].unique()))
    notclassified.append(incorrect)

In [ ]:
fig,ax = plt.subplots(figsize=(20,10))

ax.plot(threshholds,
        accuracy,
        color="red", 
        marker="o")
# set x-axis label
ax.set_xlabel("threshold", fontsize = 14)
# set y-axis label
ax.set_ylabel("accuracy",
              color="red",
              fontsize=14)

ax.set_xticks(list(threshholds))
ax2=ax.twinx()
ax2.plot(threshholds, notclassified,color="blue",marker="o")
ax2.set_ylabel("notclassified",color="blue",fontsize=14)
plt.grid(color = 'green', linestyle = '--', linewidth = 0.5)
plt.show()

In [ ]:
checkfor = range(len(label_dict))
#checkfor = [0]
final_predicted = []

for a,p, c in zip(actual, predicted, cr):
    if c>0.90:
        final_predicted.append(p)
    else:
        final_predicted.append(0)
    

for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p, c in zip(actual, final_predicted, cr):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            #elif ((p!=check)&(c<0.90)):
            #    correct+=1
            #elif ((p!=check)&(c>0.90)):
            #    notclassified+=1
            elif (p!=check):
                notclassified+=1
                score.append(c)
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_dict_inv[check],correct,total,acc))
        print('notclassified {}'.format(notclassified))
        if notclassified!=0:
            print('notclassified score for {}'.format(np.median(score)))
    print('\n')

In [ ]:
global_validate['predlabel'] = list(final_predicted)
global_validate['predicted'] = [label_dict_inv[i] for i in final_predicted]
global_validate['cr'] = list(cr)
#global_validate.head(2)

In [ ]:
global_validate['category'][(global_validate['new_category']=='others')&
                            (global_validate['predicted']!='others')].value_counts()

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate[(global_validate['label']==0)&(global_validate['predlabel']!=0)].shape

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
global_validate.shape

### after deployment validation

In [ ]:
#global_validate = pd.read_csv('/data/mmortgage/amal_workspace/multi_page_classification_preprocessing/domain_testing/BERT Testing_10.08.2022_Results.csv')
global_validate = pd.read_csv('./test.csv')
global_validate.head(1)
global_validate.shape

In [ ]:
# count=0
# index=[]
# for i,adr in enumerate(global_validate['ADR']):
#     if adr in data['ADR'].values.tolist():
#         count+=1
#         index.append(i)
# global_validate.drop(index=index, inplace=True)
global_validate.shape

In [ ]:
global_validate['Actual Classification'].value_counts()

In [ ]:
label = {
    'others': 0,
 'Retirement Account Statement(s)': 1,
 'Mortgage Statement': 2,
 'Bank Statement': 3,
 'Non-Mortgage Statement': 4,
 'Due Diligence Report': 5,
 'Purchase Agreement': 6,
 'Rental Agreements(s)': 7,
 'Hazard Insurance': 8,
 'Divorce Decree / Child Support': 9,
 'Bankruptcy Papers': 10
}

In [ ]:
label_inv=dict()
for lab in label:
    label_inv[label[lab]]=lab
label_inv 

In [ ]:
temp = []

cats = list(label.keys())

for category in global_validate['Actual Classification']:
    if category not in cats:
        temp.append('others')
    else:
        temp.append(category)
global_validate['new_category'] = temp

In [ ]:
global_validate['label'] = global_validate.new_category.replace(label)
global_validate.head(2)

In [ ]:
global_validate['new_category'].value_counts()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, 
                 dropout=0.1, 
                 BERTconfig=False, 
                 attention_heads=12, 
                 hidden_layers=12, 
                 numclass=1,
                 hidden_dropout_prob=0.1,
                 hidden_act='gelu',
                 position_embedding_type='absolute'):
        
        #self.attention_heads = attention_heads
        super(BertClassifier, self).__init__()
        
        if BERTconfig:
            configuration = BertConfig(num_attention_heads= attention_heads, 
                                       num_hidden_layers= hidden_layers)   
        else:
            configuration = BertConfig()
        #self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=attention)
        #print(configuration)
        self.bert = BertModel(configuration)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, numclass, bias=True)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)

        return linear_output

In [ ]:
import configparser
import ast

device = torch.device('cuda')

configs = configparser.ConfigParser()
configs.read('./config.ini')
labels = []
BERT_MODELS = []
THRESHOLDS=[]
GARBAGE_REMOVAL=[]

for config in configs.sections():
    label_ = ast.literal_eval(configs[config].get('categoryMAP'))
    labels.append(label_)
    THRESHOLDS.append(float(configs[config].get('cr_thresh')))
    GARBAGE_REMOVAL.append(bool(configs[config].get('garbageremoval')))
    HIDDEN_LAYERS = int(configs[config].get('hidden_layers'))
    ATTENTIONS = int(configs[config].get('attentions'))
    CLASSES = len(label_)
    MODEL_FOLDER = configs[config].get('modelfolder')
    model = BertClassifier(BERTconfig=True, 
                           attention_heads=ATTENTIONS, 
                           hidden_layers=HIDDEN_LAYERS, 
                           numclass= CLASSES)
    model.to(device)
    model.load_state_dict(torch.load('./model/{}/BERT.model'.format(MODEL_FOLDER), map_location=torch.device('cuda')))
    model.eval()
    BERT_MODELS.append(model)

In [ ]:
remove_words = set(stopwords.words('english') + list(string.punctuation))
def predict(model, text, labels, thresh, grabageremoval):
    
    if grabageremoval:
        text = re.sub(r'[~^0-9]', '', str(text))
        wordlist = [word for word in word_tokenize(text.lower()) if not word in remove_words]
        text = (" ").join(wordlist)
    
    encoded_data = tokenizer.encode_plus(
            str(text), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding='max_length', 
            max_length=512, 
            truncation=True,
            return_tensors='pt')
    
    input_ids_text = encoded_data['input_ids'].to(device)
    attention_masks_text = encoded_data['attention_mask'].to(device)

    with  torch.no_grad(): 
        outputs = model(input_ids_text, attention_masks_text)
        
    scores = torch.nn.functional.softmax(outputs.detach().cpu(), dim=1).numpy()[0]
    
    if np.max(scores)>thresh:
        return (labels[np.argmax(scores)],np.max(scores))
    else:
        return ('others', np.max(scores))

In [ ]:
txts = ['LibertyGuard Deluxe Homeowner Policy Declarations Liberty Mutual Personal Insurance Company Liberty Mutual. INSURANCE FAX: ATTN: POLICY NUMBER: H3V-251-492955-70 NAME & ADDRESS Charles Corbett Dolores McKeehan 403 Poplar Ridge Rd Chapmansboro, TN 37035-5334 RESIDENCE PREMISES INSURED 4127 Meadow View Cir Pleasant View, TN 37146-8198 THESE DECLARATIONS EFFECTIVE 12/21/2021 Same as Residence POLICY PERIOD 12/21/2021 through 12/21/2022 RESIDENCE PREMISES 403 Poplar Ridge Rd Chapmmsboro, TN 37035-5334 SECTIONI AND II: COVERAGES AND LIMITS UNDER YOUR LIBERTY GUARD HOMEOWNERS POLICY I: COVERAGE A - YOUR DWELLING COVERAGE B - OTHER STRUCTURES ON RESIDENCE PREMISES COVERAGE C - PERSONAL PROPERTY COVERAGE D - LOSS OF USE OF YOUR RESIDENCE PREMISES $ 280,500 $ 28,050 $ 210,380 Actual Loss Sustained II: COVERAGE E - PERSONAL LIABILITY (EACH OCCURRENCE) COVERAGE F - MEDICAL PAYMENTS TO OTHERS (EACH PERSON) $ 300,000 5,000 DEDUCTIBLE: LOSSES COVERED UNDER SECTION I ARE SUBJECT TO A DEDUCTIBLE OF 1% Wind/Hail (if applicable) 1% NET PREMIUM: $1,359.00 PAID IN FULL NO Replacement Cost Coverage X Yes No Expanded Replacement Cost [X] 20% No Functional Replacement Roof Replacement Cost Coverage Yes [X] No Mortgagee 1 HIGHLANDS RESIDENTIAL MORTGAGE, LTD. Loan # 7018195910 Issoe/atime C/O Conlar P.O. Box 202028 Florence, SC 29502 Jans mathe Market President Secretary Countersigned by: Date: December 09, 2021']

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

start_time = time.time()

pred_label = []
i=0
for txt in global_validate['text'].values:
#for txt in txts:
#for txt in zip(global_validate['text'].values, global_validate['Ref. No.'].values, global_validate['text'].values:
    i=i+1
    print(i)
    with ThreadPoolExecutor() as executor:
        predictions = executor.map(predict, 
                                   BERT_MODELS, 
                                   [txt]*len(labels), 
                                   labels, 
                                   THRESHOLDS,
                                   GARBAGE_REMOVAL)
    predictions = list(predictions)
    print(predictions)

    if(predictions[0][0]=='focused'):
        classifications = list(filter(lambda x: x[0]!='others', predictions[1:]))
        if len(classifications)==0:
            final_class = sorted(predictions[1:],key=lambda x: x[1], reverse=True)[0]
        else:
            final_class = sorted(classifications,key=lambda x: x[1], reverse=True)[0]

        '''if final_class[1]>0.90:
            final_class = final_class
        else:
            final_class = ('others',final_class[1])'''
    else:
        final_class = ('others',predictions[0][1])
        
    pred_label.append(label[final_class[0]])
    print(final_class)
    print('\n')
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
label

In [ ]:
acc = accuracy_score(global_validate['label'].values, np.array(pred_label))
f1 = f1_score(global_validate['label'].values, np.array(pred_label), average='weighted')
acc,f1

In [ ]:
label_inv

In [ ]:
range(len(label_inv))

In [ ]:
checkfor = range(len(label_inv))
for check in checkfor:
    total = 0
    correct = 0
    notclassified = 0
    score = []
    for a,p in zip(global_validate['label'].values, np.array(pred_label)):
             
        if a == check:
            total+=1
            if (p == check):
                correct+=1
            elif (p!=check):
                notclassified+=1
                
    if total!=0:
        acc = round(correct/total,2)
        print('accuracy for {}: {}/{}={}'.format(label_inv[check],correct,total,acc))
        #print('notclassified {}'.format(notclassified))
        print('\n')

In [ ]:
global_validate['pred_label'] = pred_label
global_validate.head(1)

In [ ]:
global_validate['pred_category'] = global_validate['pred_label'].replace(label_inv )
global_validate.head(1)

In [ ]:
global_validate['new_category'].values

In [ ]:
temp = []
for i,j in zip(global_validate['new_category'].values, global_validate['pred_category'].values):
    if i==j:
        temp.append('PASS')
    else:
        temp.append('FAIL')
global_validate['local_P/F']= temp

In [ ]:
global_validate.to_csv('./valid.csv', header=True, index=False)

In [ ]:
global_validate[(global_validate['label']==0)&
                            (global_validate['pred_label']!=0)].shape

In [ ]:
bank 
morgage 
Retirement

In [ ]:
Purchase
Rental

In [ ]:
tax return
k1
w2
paystub

In [ ]:
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac

In [ ]:
augc= nac.OcrAug() 
'''aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', 
                                model_type='bert',
                                action="insert", 
                                aug_p=0.80)'''
                                
#augs = nas.ContextualWordEmbsForSentenceAug(model_path='gpt2', top_p=0.5, batch_size=4)
#randomSentAug = nas.random.RandomSentAug(mode='neighbor', action='swap') <---

In [ ]:
txt = [data['text'][1],data['text'][0]]
augmented_data = augc.augment(txt)
#augmented_data = randomWordAug.augment(txt)

In [ ]:
txt

In [ ]:
augmented_data